## Homework 4: Accumulation Opportunity In Electronic Market
### FINM 33150 (Winter 2025) - Quantitative Trading Strategies
#### NAME: Arturo Charleston
#### Student ID: 12449584

In [4]:
# PACKAGES
import os
import sys
import warnings
import re
import pandas as pd
import numpy as np
import h5py

# Global Options
warnings.filterwarnings("ignore")
pd.options.display.float_format = '{:.5f}'.format

# Set directory
home_directory = "/Users/charleston/Downloads/git_repositories/finm-qts-2025"
os.chdir(home_directory)




In [7]:
search_path = os.path.join(home_directory, 'data', 'Crypto')
list_of_files = []

for year in [2022, 2023]:
    folder = os.path.join(search_path, str(year))
    for filename in os.listdir(folder):
        if "narrow" in filename.lower():
            match = re.search(r"(book|trades)_narrow_([^_]+)_(\d{4})", filename)
            if match:
                data_type = match.group(1) 
                symbols = match.group(2).replace('-', '_')
                df_name = f"{symbols}_{data_type}_{year}"
                filepath = os.path.join(folder, filename)
                print(f"Reading {filepath} into {df_name}")
                list_of_files.append(df_name)
                if filename.endswith(".gz"):
                    globals()[df_name] = pd.read_csv(filepath, compression='gzip', sep='\t')
                else:
                    globals()[df_name] = pd.read_csv(filepath, sep='\t')


# For 2024
year = 2024
folder_2024 = os.path.join(search_path, str(year))
for filename in os.listdir(folder_2024):
    match = re.search(r"2024-\d+__([^_]+)_(orders|trades)", filename)
    if match:
        symbols = match.group(1).replace('-', '_')
        if match.group(2) == "orders":
            data_type = "book"
        else:
            data_type = "trades"

        filepath = os.path.join(folder_2024, filename)
        if filename.endswith(".h5"):
            with h5py.File(filepath, "r") as f:
                if "Level2" in f:
                    level2_group = f["Level2"]
                    for dataset_name in level2_group.keys():
                        data_array = level2_group[dataset_name][:]
                        if data_array.dtype.names:
                            df = pd.DataFrame(data_array, columns=data_array.dtype.names)
                        else:
                            df = pd.DataFrame(data_array)
                        ds_symbols = dataset_name.replace('-', '_')
                        df_name = f"{ds_symbols}_{data_type}_{year}"
                        print(f"Creating DataFrame {df_name}")
                        list_of_files.append(df_name)
                        globals()[df_name] = df

Reading /Users/charleston/Downloads/git_repositories/finm-qts-2025/data/Crypto/2022/book_narrow_ETH-BTC_2022.delim.gz into ETH_BTC_book_2022
Reading /Users/charleston/Downloads/git_repositories/finm-qts-2025/data/Crypto/2022/trades_narrow_ETH-USD_2022.delim.gz into ETH_USD_trades_2022
Reading /Users/charleston/Downloads/git_repositories/finm-qts-2025/data/Crypto/2022/trades_narrow_ETH-BTC_2022.delim.gz into ETH_BTC_trades_2022
Reading /Users/charleston/Downloads/git_repositories/finm-qts-2025/data/Crypto/2022/trades_narrow_BTC-USD_2022.delim.gz into BTC_USD_trades_2022
Reading /Users/charleston/Downloads/git_repositories/finm-qts-2025/data/Crypto/2023/book_narrow_ETH-BTC_2023.delim.gz into ETH_BTC_book_2023
Reading /Users/charleston/Downloads/git_repositories/finm-qts-2025/data/Crypto/2023/book_narrow_BTC-USD_2023.delim.gz into BTC_USD_book_2023
Reading /Users/charleston/Downloads/git_repositories/finm-qts-2025/data/Crypto/2023/trades_narrow_ETH-USD_2023.delim.gz into ETH_USD_trades_20

In [9]:
BTC_USD_book_2023

,Ask1PriceMillionths,Bid1PriceMillionths,Ask1SizeBillionths,Bid1SizeBillionths,Ask2PriceMillionths,Bid2PriceMillionths,Ask2SizeBillionths,Bid2SizeBillionths,received_utc_nanoseconds,timestamp_utc_nanoseconds,Mid
0,22972550000,22970150000,210000000,25797600,22972560000,22970120000,210000000,87069610,1674521267750919800,1674521267806932000,22971350000.00000
1,22972550000,22970150000,410000000,25797600,22972560000,22970120000,210000000,87069610,1674521267751154000,1674521267807073000,22971350000.00000
2,22972550000,22970150000,410000000,25797600,22972560000,22970120000,210000000,87069610,1674521267752363000,1674521267813516000,22971350000.00000
3,22972540000,22970150000,210000000,25797600,22972550000,22970120000,410000000,87069610,1674521267763308000,1674521267819272000,22971345000.00000
4,22972540000,22970150000,210000000,25797600,22972550000,22970120000,410000000,87069610,1674521267764161000,1674521267825538000,22971345000.00000
...,...,...,...,...,...,...,...,...,...,...,...
10700407,22571340000,22569880000,85809220,100000000,22571350000,22569440000,155580,200000000,1674661801846230000,1674661801834922000,22570610000.00000
10700408,22571340000,22569880000,85809220,100000000,22571350000,22569440000,12801710,200000000,1674661801847329100,1674661801846675000,22570610000.00000
10700409,22571340000,22569880000,85809220,100000000,22571350000,22569440000,12801710,200000000,1674661801857751100,1674661801855921000,22570610000.00000
10700410,22571330000,22569880000,88607990,100000000,22571340000,22569440000,85809220,200000000,1674661801881766800,1674661801889084000,22570605000.00000


In [10]:
BTC_USD_trades_2022

,received_utc_nanoseconds,timestamp_utc_nanoseconds,PriceMillionths,SizeBillionths,Side
0,1643490176220148000,1643490169574850000,38383380000,938300,1.00000
1,1643490176220541000,1643490169574850000,38387760000,8937830,1.00000
2,1643490176220914000,1643490169574850000,38387770000,46454510,1.00000
3,1643490176221462000,1643490169574850000,38388960000,14455400,2.00000
4,1643490176221856000,1643490169574850000,38390720000,33673620,1.00000
...,...,...,...,...,...
2907371,1644024594763098000,1644024594843404000,41395800000,280760,1.00000
2907372,1644024594763689000,1644024594843404000,41395820000,284330,1.00000
2907373,1644024595042286000,1644024595024425000,41400780000,2062590,1.00000
2907374,1644024595111889000,1644024595195843000,41400290000,1106040,1.00000
